In [1]:
#import csv
import json
import requests
import pandas as pd
from tqdm import tqdm  # progress_apply
import postgis as pg
from pymeos import *
import contextily as cx
# from tqdm.notebook import tqdm
import asyncio
import aiohttp
# -----------------------
# CONFIGURATION
# -----------------------
SERVER_URL = "http://localhost:8080"
COLLECTION_ID = "ships"           # Change to your collection name


LIMIT_ROWS = None
BATCH_SIZE = 50


/home/sirine/Desktop/pg_mfserv/env/lib/python3.13/site-packages/postgis/__init__.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# based on PYMEOS example : https://github.com/MobilityDB/PyMEOS-Examples/blob/main/PyMEOS_Examples/AIS.ipynb
# -----------------------
# HELPER FUNCTIONS
# -----------------------

# rework the data using pandas and pymeos to get temporal geometry (trip)

In [2]:
tqdm.pandas()

pymeos_initialize()
# %%time
# get data, cleanup remove duplicates, wrong lat,long ///



ais = pd.read_csv(
    "./data/aisdk-2024-03-01.zip",
    usecols=["# Timestamp", "MMSI", "Latitude", "Longitude", "SOG"],
)
ais.columns = ["t", "mmsi", "lat", "lon", "sog"]
# data preparation:  remove rows with no timestamps
ais = ais[ais["t"] != 0]
ais["t"] = pd.to_datetime(ais["t"], format='%d/%m/%Y %H:%M:%S')
ais = ais[ais["mmsi"] != 0]
ais = ais.drop_duplicates(["t", "mmsi"])
ais = ais[(ais["lat"] >= 40.18) & (ais["lat"] <= 84.17)]
ais = ais[(ais["lon"] >= -16.1) & (ais["lon"] <= 32.88)]
ais = ais[(ais["sog"] >= 0) & (ais["sog"] <= 1022)]
ais.dropna()
ais.head()

,t,mmsi,lat,lon,sog
0,2024-03-01,219000873,56.990910,10.304543,0.0
2,2024-03-01,219016683,56.800165,9.024933,0.0
3,2024-03-01,219000615,56.967093,9.224287,2.3
5,2024-03-01,219017664,56.974950,8.922530,0.0
6,2024-03-01,219002686,56.795143,8.863960,0.0


# Now, we will create the PyMEOS object representing
# the position and the SOG.

In [3]:
ais["instant"] = ais.progress_apply(
    lambda row: TGeogPointInst(
        point=(row["lon"], row["lat"]), timestamp=row["t"]),
    axis=1,
)
ais["sog"] = ais.progress_apply(
    lambda row: TFloatInst(value=row["sog"], timestamp=row["t"]), axis=1
)
# dropt the standard type columns
ais.drop(["lat", "lon"], axis=1, inplace=True)
ais.head()

100%|██████████| 9183632/9183632 [01:37<00:00, 93804.30it/s] 


,t,mmsi,sog,instant
0,2024-03-01,219000873,0@2024-03-01 00:00:00+01,POINT(10.304543 56.99091)@2024-03-01 00:00:00+01
2,2024-03-01,219016683,0@2024-03-01 00:00:00+01,POINT(9.024933 56.800165)@2024-03-01 00:00:00+01
3,2024-03-01,219000615,2.3@2024-03-01 00:00:00+01,POINT(9.224287 56.967093)@2024-03-01 00:00:00+01
5,2024-03-01,219017664,0@2024-03-01 00:00:00+01,POINT(8.92253 56.97495)@2024-03-01 00:00:00+01
6,2024-03-01,219002686,0@2024-03-01 00:00:00+01,POINT(8.86396 56.795143)@2024-03-01 00:00:00+01


# Assembling Trips (MEOS Example)
# Now, we will create the trajectory (TGeogPointSeq) and
# the SOG evolution (TFloatSeq) for every ship (identified by the mmsi)
# using the instants we have created.
# ::::::::::::::::::::::::::::stopped here , time limit

In [4]:

trajectories = (
    ais.groupby("mmsi")
    .aggregate(
        {
            "instant": lambda x: TGeogPointSeq.from_instants(x, upper_inc=True),
            "sog": lambda x: TFloatSeq.from_instants(x, upper_inc=True),
        }
    )
    .rename({"instant": "trajectory"}, axis=1)
)
trajectories["distance"] = trajectories["trajectory"].apply(
    lambda t: t.length())
trajectories.head()

,trajectory,sog,distance
mmsi,,,
174,[POINT(8.71036 53.86317)@2024-03-01 04:54:01+01],[0.1@2024-03-01 04:54:01+01],0.000000
5322,[POINT(12.606015 55.684605)@2024-03-01 00:00:0...,"[0@2024-03-01 00:00:09+01, 0@2024-03-01 00:02:...",31316.141996
2190045,[POINT(8.448052 55.464242)@2024-03-01 00:00:11...,"[0@2024-03-01 00:00:11+01, 0@2024-03-01 01:18:...",4305.838535
97000050,[POINT(9.56149 57.652025)@2024-03-01 06:56:26+...,"[1.9@2024-03-01 06:56:26+01, 0.9@2024-03-01 06...",56.898672
111211505,[POINT(9.916197 53.931403)@2024-03-01 08:48:11...,"[89@2024-03-01 08:48:11+01, 131@2024-03-01 08:...",397413.458054


# ::::::::::::::::::::::::::::::stopped here , time limit
# Storing in MobilityDB: so they use PYMEOS but here use server.py- susceptible d'etre modifié!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# https://stackoverflow.com/questions/51699817/python-async-post-requests


In [ ]:
import requests
import json

# Clean up resources after the operation (if needed)
def clean_up_resources():
    # If you have resources to clean up or finalize like MEOS, you can call it here
    try:
        pymeos_finalize()  # Finalize MEOS operations, ensure this is needed
        print("MEOS resources cleaned up successfully.")
    except Exception as e:
        print(f"Error during resource cleanup: {str(e)}")

# Base URL of the server
server_url = 'http://localhost:8080/collections/{collectionId}/items'

# You need to replace `{collectionId}` with the actual collection ID you want to post to
collection_id = 'ships'
trajectories= trajectories.iloc[:2]
# Iterate over the DataFrame and prepare each item for posting
for index, row in trajectories.iterrows():
    try:
        # Prepare the data
        post_data = {
            "type":"Feature",
            "id": row.name,  # Assuming the index of the dataframe is the feature ID
            "temporalGeometry": row["trajectory"].as_mfjson()  # Convert trajectory (TGeogPointSeq) to mfjson
        }
        print("fjkdfkjkjh")
        print(row["trajectory"].as_mfjson() )
        # Convert the dictionary to a JSON string
        json_data = json.dumps(post_data)

        # Make the POST request to the server
        response = requests.post(
            server_url.format(collectionId=collection_id), 
            data=json_data, 
            headers={'Content-Type': 'application/json'}
        )

        # Check the response from the server
        if response.status_code == 200:
            print(f"Successfully posted feature {row.name}")
        else:
            print(f"Failed to post feature {row.name}: {response.status_code} - {response.text}")
    
    except requests.exceptions.RequestException as e:
        # Handle any request-related exceptions (timeouts, connection issues, etc.)
        print(f"Request error for feature {row.name}: {str(e)}")
    
    except Exception as e:
        # Handle any other exceptions
        print(f"Unexpected error for feature {row.name}: {str(e)}")

# Clean up resources after the operation (e.g., MEOS library)
clean_up_resources()


fjkdfkjkjh
{
  "type": "MovingPoint",
  "bbox": [
    [
      8.71036,
      53.86317
    ],
    [
      8.71036,
      53.86317
    ]
  ],
  "period": {
    "begin": "2024-03-01T04:54:01+01",
    "end": "2024-03-01T04:54:01+01",
    "lower_inc": true,
    "upper_inc": true
  },
  "coordinates": [
    [
      8.71036,
      53.86317
    ]
  ],
  "datetimes": [
    "2024-03-01T04:54:01+01"
  ],
  "lower_inc": true,
  "upper_inc": true,
  "interpolation": "Linear"
}
Failed to post feature 174: 500 - {"code": "500", "description": "MeosMfJsonInputError (20): Unable to find 'type' in MFJSON string"}
fjkdfkjkjh
{
  "type": "MovingPoint",
  "bbox": [
    [
      12.603643,
      55.683407
    ],
    [
      12.607205,
      55.686422
    ]
  ],
  "period": {
    "begin": "2024-03-01T00:00:09+01",
    "end": "2024-03-01T23:59:49+01",
    "lower_inc": true,
    "upper_inc": true
  },
  "coordinates": [
    [
      12.606015,
      55.684605
    ],
    [
      12.605993,
      55.684613
    ],
